In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import linear_model# import LogisticRegression
from subprocess import check_output

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

gender_submission.csv
test.csv
train.csv



Load and merge datasets

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:

data = train.append(test, sort=False)
data.count()

PassengerId    1309
Survived        891
Pclass         1309
Name           1309
Sex            1309
Age            1046
SibSp          1309
Parch          1309
Ticket         1309
Fare           1308
Cabin           295
Embarked       1307
dtype: int64

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
def fillna_mean(feature: pd.Series):
    mean = feature.mean()
    feature.fillna(mean, inplace=True)
    
fillna_mean(data['Fare'])
fillna_mean(data['Age'])

In [6]:
# new features - FamilySize
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

In [7]:
# new features - Alone
data['Alone'] = 0
data.loc[data['FamilySize'] == 1, 'Alone'] = 1

In [8]:
# extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ''

# new feature - Title
data['Title'] = data['Name'].apply(get_title)
data['Title'].replace('Mlle', 'Miss', inplace=True)
data['Title'].replace('Ms', 'Miss', inplace=True)
data['Title'].replace('Mme', 'Mrs', inplace=True)
data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare', inplace=True)

In [9]:
def encode_labels(label: str):
    le = LabelEncoder()
    data[label].fillna('', inplace=True)
    le.fit(data[label])
    data.loc[:, label] = le.transform(data[label])

encode_labels('Sex')
encode_labels('Embarked')
encode_labels('Title')

In [10]:
train = data[data['Survived'].notna()]
test = data[data['Survived'].isna()]

In [11]:
# prepare train for fitting
# other features appears useless
features = ['Age', 'Sex', 'Pclass', 'Fare', 'Parch', 'Alone']
X = train.loc[:, features]
Y = train.loc[:, 'Survived']
XTrain, XValid, YTrain, YValid = train_test_split(X, Y, test_size=0.2, random_state=40)

In [12]:
# Fit logistic regression using scikit
LR = linear_model.LogisticRegression(C=1000, solver='lbfgs', max_iter=1000)
LR.fit(X=XTrain, y=YTrain)

def accuracy(Y: np.array, yPred: np.array) -> float:
  return np.sum(yPred==Y) / len(Y)

# Use model to predict on training and validation sets
print('     Train accuracy', accuracy(YTrain, LR.predict(XTrain)))
print('Validation accuracy', accuracy(YValid, LR.predict(XValid)))

     Train accuracy 0.7963483146067416
Validation accuracy 0.8268156424581006


In [13]:
# Predict for test set
# Create a Kaggle submission
XTest = test.loc[:, features]
YTest = LR.predict(XTest)
#data['Survivd'] = data['Survived'].astype('int64')
sub = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': YTest.astype('int64')})
sub.to_csv('submission.csv', index=False)